<a href="https://colab.research.google.com/github/satya-ssp/JupyterNotebooks-Medium/blob/master/R6_Internal_Lab_UpdatedTF2_Prices_Iris.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Linear Classifier in TensorFlow 
Using Low Level API in Eager Execution mode

### Load tensorflow

In [0]:
import tensorflow as tf

In [0]:
#Enable Eager Execution if using tensflow version < 2.0
#From tensorflow v2.0 onwards, Eager Execution will be enabled by default


### Collect Data

In [35]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import pandas as pd

In [0]:
data_pricing = pd.read_csv('/content/drive/My Drive/AIML 2019/prices.csv')

### Check all columns in the dataset

In [42]:
data_pricing.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 851264 entries, 0 to 851263
Data columns (total 7 columns):
date      851264 non-null object
symbol    851264 non-null object
open      851264 non-null float64
close     851264 non-null float64
low       851264 non-null float64
high      851264 non-null float64
volume    851264 non-null float64
dtypes: float64(5), object(2)
memory usage: 45.5+ MB


### Drop columns `date` and  `symbol`

In [0]:
data_pricing_new = data_pricing.drop(['date','symbol'],axis=1)

In [44]:
data_pricing_new.head()

,open,close,low,high,volume
0,123.430000,125.839996,122.309998,126.250000,2163600.0
1,125.239998,119.980003,119.940002,125.540001,2386400.0
2,116.379997,114.949997,114.930000,119.739998,2489500.0
3,115.480003,116.620003,113.500000,117.440002,2006300.0
4,117.010002,114.970001,114.089996,117.330002,1408600.0


### Consider only first 1000 rows in the dataset for building feature set and target set
Target 'Volume' has very high values. Divide 'Volume' by 1000,000

data['volume'] = data['volume'] / 1000000

In [0]:
data_pricing_new['volume'] = data_pricing_new['volume'].apply(lambda x: x/1000000)

In [0]:
pricing = data_pricing_new.head(1000)

### Divide the data into train and test sets

In [0]:
from sklearn.model_selection import train_test_split

X =  pricing.drop("volume", axis=1)
y =  pricing.pop("volume")
Xtrain,Xtest,ytrain,ytest = train_test_split(X, y, test_size=0.30, random_state=1)

#### Convert Training and Test Data to numpy float32 arrays


import numpy as np
x = np.float32

In [0]:
import numpy as np
Xtrain = np.asarray(Xtrain)
ytrain = np.asarray(ytrain)

Input features
x = tf.placeholder(shape=[None,13],dtype=tf.float32, name='x-input')

### Normalize the data
You can use Normalizer from sklearn.preprocessing

Normalize the data
x_n = tf.nn.l2_normalize(x,1)

Actual Prices
y_ = tf.placeholder(shape=[None],dtype=tf.float32, name='y-input')

In [0]:
from sklearn import preprocessing
X_normalized = preprocessing.normalize(X, norm='l2')

## Building the Model in tensorflow

1.Define Weights and Bias, use tf.zeros to initialize weights and Bias

In [0]:
#Input features
x = tf.placeholder(shape=[None,4],dtype=tf.float32, name='x-input')

#Normalize the data
x_n = tf.nn.l2_normalize(x,1) 

#Actual Prices
y_ = tf.placeholder(shape=[None],dtype=tf.float32, name='y-input')

W = tf.Variable(tf.zeros(shape=[4,1]), name="Weights")
b = tf.Variable(tf.zeros(shape=[1]),name="Bias")

2.Define a function to calculate prediction

In [0]:
y = tf.add(tf.matmul(x_n,W),b,name='output') #X is normalized, hence X_n

3.Loss (Cost) Function [Mean square error]

In [0]:
loss = tf.reduce_mean(tf.square(y-y_),name='Loss')

4.Function to train the Model

1.   Record all the mathematical steps to calculate Loss
2.   Calculate Gradients of Loss w.r.t weights and bias
3.   Update Weights and Bias based on gradients and learning rate to minimize loss

In [0]:
train_op = tf.train.GradientDescentOptimizer(0.03).minimize(loss)

## Train the model for 100 epochs 
1. Observe the training loss at every iteration
2. Observe Train loss at every 5th iteration

In [0]:
#Lets start graph Execution
sess = tf.Session()

# variables need to be initialized before we can use them
sess.run(tf.global_variables_initializer())

#how many times data need to be shown to model
training_epochs = 100

### Get the shapes and values of W and b

In [55]:
print(W.shape)
print(b.shape)

(4, 1)
(1,)


### Model Prediction on 1st Examples in Test Dataset

In [56]:
for epoch in range(training_epochs):
            
    #Calculate train_op and loss
    _, train_loss = sess.run([train_op,loss],feed_dict={x:Xtrain, y_:ytrain}) # X and Y is like X_Train and Y_Train here.
    
    if epoch % 5 == 0:
        print ('Training loss at step: ', epoch, ' is ', train_loss)

('Training loss at step: ', 0, ' is ', 225.87958)
('Training loss at step: ', 5, ' is ', 203.89143)
('Training loss at step: ', 10, ' is ', 197.76749)
('Training loss at step: ', 15, ' is ', 196.0619)
('Training loss at step: ', 20, ' is ', 195.58687)
('Training loss at step: ', 25, ' is ', 195.45459)
('Training loss at step: ', 30, ' is ', 195.41774)
('Training loss at step: ', 35, ' is ', 195.4075)
('Training loss at step: ', 40, ' is ', 195.40462)
('Training loss at step: ', 45, ' is ', 195.40382)
('Training loss at step: ', 50, ' is ', 195.4036)
('Training loss at step: ', 55, ' is ', 195.40355)
('Training loss at step: ', 60, ' is ', 195.40353)
('Training loss at step: ', 65, ' is ', 195.40355)
('Training loss at step: ', 70, ' is ', 195.40353)
('Training loss at step: ', 75, ' is ', 195.40353)
('Training loss at step: ', 80, ' is ', 195.40353)
('Training loss at step: ', 85, ' is ', 195.40353)
('Training loss at step: ', 90, ' is ', 195.40355)
('Training loss at step: ', 95, ' is

In [57]:
test_loss = sess.run([loss],feed_dict={x:Xtest, y_:ytest}) 
print ('Training loss at step:is ', test_loss)

('Training loss at step:is ', [241.01949])


## Classification using tf.Keras

In this exercise, we will build a Deep Neural Network using tf.Keras. We will use Iris Dataset for this exercise.

### Load the given Iris data using pandas (Iris.csv)

In [0]:
import tensorflow as tf
import pandas as pd
data = pd.read_csv('/content/drive/My Drive/AIML 2019/11_Iris.csv')

### Target set has different categories. So, Label encode them. And convert into one-hot vectors using get_dummies in pandas.

In [0]:
k_data = pd.get_dummies(data,prefix=['Species'], columns=['Species'])

In [62]:
k_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 8 columns):
Id                         150 non-null int64
SepalLengthCm              150 non-null float64
SepalWidthCm               150 non-null float64
PetalLengthCm              150 non-null float64
PetalWidthCm               150 non-null float64
Species_Iris-setosa        150 non-null uint8
Species_Iris-versicolor    150 non-null uint8
Species_Iris-virginica     150 non-null uint8
dtypes: float64(4), int64(1), uint8(3)
memory usage: 6.4 KB


train_small_with_dummies = pd.get_dummies(train_small, sparse=True)

### Splitting the data into feature set and target set

In [0]:
from sklearn.model_selection import train_test_split

X =  k_data.drop(['Id',"Species_Iris-setosa",'Species_Iris-versicolor',
       'Species_Iris-virginica'], axis=1)
y =  k_data[['Species_Iris-setosa', 'Species_Iris-versicolor',
       'Species_Iris-virginica']]
Xtrain,Xtest,ytrain,ytest = train_test_split(X, y, test_size=0.30, random_state=1)

(features, actual_prices),_ = tf.keras.datasets.boston_housing.load_data(test_split=0)

###  Building Model in tf.keras

Build a Linear Classifier model  <br>
1.  Use Dense Layer  with input shape of 4 (according to the feature set) and number of outputs set to 3<br> 
2. Apply Softmax on Dense Layer outputs <br>
3. Use SGD as Optimizer
4. Use categorical_crossentropy as loss function 

In [64]:
from sklearn.neural_network import MLPClassifier
model=MLPClassifier(hidden_layer_sizes=(4))
model.fit(Xtrain,ytrain)

/usr/local/lib/python2.7/dist-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=4, learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=None, shuffle=True, solver='adam', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False)

#Initialize Sequential Graph (model)
model = tf.keras.models.Sequential()

#Normalize input data
model.add(tf.keras.layers.BatchNormalization(input_shape=(13,)))

#Add Dense layer for prediction - Keras declares weights and bias automatically
model.add(tf.keras.layers.Dense(1))

#Compile the model - add Loss and Gradient Descent optimizer
model.compile(optimizer='sgd', loss='mse')

### Model Training 

In [65]:
from keras.models import Sequential
from keras.layers import Dense

model = Sequential()
model.add(Dense(3, input_dim=4, kernel_initializer='normal', activation='softmax'))
#model.add(Dense(3, kernel_initializer='normal'))
# Compile model
model.compile(loss='categorical_crossentropy', optimizer='SGD', metrics= ['accuracy'])

Using TensorFlow backend.
W0721 12:38:45.026407 139666332325760 deprecation_wrapper.py:119] From /usr/local/lib/python2.7/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0721 12:38:45.028867 139666332325760 deprecation_wrapper.py:119] From /usr/local/lib/python2.7/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0721 12:38:45.034034 139666332325760 deprecation_wrapper.py:119] From /usr/local/lib/python2.7/dist-packages/keras/backend/tensorflow_backend.py:4115: The name tf.random_normal is deprecated. Please use tf.random.normal instead.

W0721 12:38:45.046658 139666332325760 deprecation_wrapper.py:119] From /usr/local/lib/python2.7/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0721 12:38:45.063688 1396663323

### Model Prediction

In [66]:
model.fit(Xtrain, ytrain, validation_data=(Xtest,ytest), epochs=10, batch_size=5)
# evaluate the model
scores = model.evaluate(X, y)

W0721 12:39:14.799396 139666332325760 deprecation.py:323] From /usr/local/lib/python2.7/dist-packages/tensorflow/python/ops/math_grad.py:1250: where (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0721 12:39:14.838546 139666332325760 deprecation_wrapper.py:119] From /usr/local/lib/python2.7/dist-packages/keras/backend/tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.



Train on 105 samples, validate on 45 samples
Epoch 1/10
105/105 [==============================] - 0s 2ms/step - loss: 1.0331 - acc: 0.4571 - val_loss: 1.0304 - val_acc: 0.6000
Epoch 2/10
105/105 [==============================] - 0s 555us/step - loss: 0.9180 - acc: 0.6857 - val_loss: 0.9296 - val_acc: 0.6000
Epoch 3/10
105/105 [==============================] - 0s 559us/step - loss: 0.8388 - acc: 0.7429 - val_loss: 0.8087 - val_acc: 0.6000
Epoch 4/10
105/105 [==============================] - 0s 537us/step - loss: 0.7671 - acc: 0.7048 - val_loss: 0.7864 - val_acc: 0.6000
Epoch 5/10
105/105 [==============================] - 0s 545us/step - loss: 0.7309 - acc: 0.7429 - val_loss: 0.7138 - val_acc: 0.6000
Epoch 6/10
105/105 [==============================] - 0s 523us/step - loss: 0.6817 - acc: 0.7429 - val_loss: 0.6660 - val_acc: 0.9556
Epoch 7/10
105/105 [==============================] - 0s 532us/step - loss: 0.6535 - acc: 0.7810 - val_loss: 0.6558 - val_acc: 0.6222
Epoch 8/10
105/105 

In [67]:
scores[1]*100

82.00000015894572

### Save the Model

In [0]:
model.save("model.dump")

### Build and Train a Deep Neural network with 2 hidden layer  - Optional - For Practice

Does it perform better than Linear Classifier? What could be the reason for difference in performance?